In [ ]:
import requests
import pdf2image
import pytesseract
import nltk
import hashlib
from neo4j import GraphDatabase
import pandas as pd
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor
import itertools

In [ ]:
# download firest article from arxiv
import requests
import pdf2image
import pytesseract

pdf = requests.get('https://arxiv.org/pdf/2110.03526.pdf')
doc = pdf2image.convert_from_bytes(pdf.content)
# Get the article text
article = []

for page_number, page_data in enumerate(doc):
    txt = pytesseract.image_to_string(page_data).encode("utf-8")

    # Sixth page contains only references
    if page_number < 6:
        article.append(txt.decode("utf-8"))

article_txt = " ".join(article)



In [ ]:
#------------------------------------------ download second article from arxiv
import requests
import pdf2image
import pytesseract

pdf1 = requests.get('https://arxiv.org/pdf/2312.09317.pdf')
doc1 = pdf2image.convert_from_bytes(pdf1.content)

# Get the article text
article1 = []

for page_number, page_data in enumerate(doc1):
    txt1 = pytesseract.image_to_string(page_data).encode("utf-8")

    # Sixth page contains only references
    if page_number < 23 :
        article1.append(txt1.decode("utf-8"))

article_txt1 = " ".join(article1)

In [ ]:
#text processing for first article
import nltk
nltk.download('punkt')

def clean_text(text):
    """Remove section titles and figure descriptions from text"""
    clean = "\n".join([row for row in text.split("\n") if
                      (len(row.split(" "))) > 3 and not (row.startswith("(a)"))
                      and not row.startswith("Figure")])
    return clean

text = article_txt.split("INTRODUCTION")[1]
ctext = clean_text(text)

sentences = nltk.tokenize.sent_tokenize(ctext)

In [ ]:
#--------------------------------------------text processing text for second article
import nltk
nltk.download('punkt')

def clean_text(text):
    """Remove section titles and figure descriptions from text"""
    clean = "\n".join([row for row in text.split("\n") if
                      (len(row.split(" "))) > 3 and not (row.startswith("(a)"))
                      and not row.startswith("Figure")])
    return clean

text1 = article_txt1.split("Introduction")[1]
ctext1 = clean_text(text1)

sentences1 = nltk.tokenize.sent_tokenize(ctext1)

In [15]:
#NER for first article
import hashlib
def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
 """Biomedical entity linking API"""
 return requests.post(url, json={'text':str(text)}).json()

In [ ]:
#NER for first article

entity_list = []
print(len(sentences))

i=0
# The last sentence is invalid
for s in sentences[:-1]:
    print("doing sentence : " + str(i))
    entity_list.append(query_plain(s))
    i += 1

print("all calls done")
parsed_entities = []

for entities in entity_list:
    e = []
    
    # If there are no entities in the text
    if not entities.get('annotations'):
        parsed_entities.append({
            'text': entities['text'],
            'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()
        })
        continue
    
    for entity in entities['annotations']:
        other_ids = [id for id in entity['id'] if not id.startswith("BERN")]
        entity_type = entity['obj']
        entity_name = entities['text'][entity['span']['begin']:entity['span']['end']]
        
        try:
            entity_id = [id for id in entity['id'] if id.startswith("BERN")][0]
        except IndexError:
            entity_id = entity_name
        
        e.append({
            'entity_id': entity_id,
            'other_ids': other_ids,
            'entity_type': entity_type,
            'entity': entity_name
        })

    parsed_entities.append({
        'entities': e,
        'text': entities['text'],
        'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()
    })

In [18]:
#-----------------------------NER for second article
import hashlib
import requests  # Import the requests module

def query_plain1(text1, url="http://bern2.korea.ac.kr/plain"):
    """Biomedical entity linking API"""
    return requests.post(url, json={'text': str(text1)}).json()

In [ ]:
#---------------------------NER for second article
#NER for first article

entity_list1 = []
print(len(sentences1))

i=0
# The last sentence is invalid
for s1 in sentences1[:-1]:
    print("doing sentence : " + str(i))
    entity_list1.append(query_plain1(s1))
    i += 1

print("all calls done")
parsed_entities1 = []

for entities1 in entity_list1:
    e1 = []
    
    # If there are no entities in the text
    if not entities1.get('annotations'):
        parsed_entities1.append({
            'text': entities1['text'],
            'text_sha256': hashlib.sha256(entities1['text'].encode('utf-8')).hexdigest()
        })
        continue
    
    for entity1 in entities1['annotations']:
        other_ids1 = [id for id in entity1['id'] if not id.startswith("BERN")]
        entity_type1 = entity1['obj']
        entity_name1 = entities1['text'][entity1['span']['begin']:entity1['span']['end']]
        
        try:
            entity_id1 = [id for id in entity1['id'] if id.startswith("BERN")][0]
        except IndexError:
            entity_id1 = entity_name1
        
        e1.append({
            'entity_id': entity_id1,
            'other_ids': other_ids1,
            'entity_type': entity_type1,
            'entity': entity_name1
        })

    parsed_entities1.append({
        'entities': e1,
        'text': entities1['text'],
        'text_sha256': hashlib.sha256(entities1['text'].encode('utf-8')).hexdigest()
    })


In [23]:
# Prepare the Neo4j connection in the notebook
from neo4j import GraphDatabase
import pandas as pd

#host = 'bolt://44.198.180.127:7687'
#user = 'neo4j'
#password = 'rating-percentages-bears'
host = 'bolt://52.91.239.25:7687'
user = 'neo4j'
password = 'disaster-ounce-waxes'
driver = GraphDatabase.driver(host, auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
    return pd.DataFrame([r.values() for r in result], columns=result.keys())


In [ ]:
# Import the author and the article into the graph
import pandas as pd
from neo4j import GraphDatabase

# Your neo4j_query function
def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        result_list = [record.data() for record in result]
    return result_list

# Example query and parameters
query = """
MERGE (a:Author {name: $author})
MERGE (b:Article {title: $title})
MERGE (a)-[:WROTE]->(b)
"""
params = {'title': '{title: $title}', 'author': '{name: $author}'}

# Fetch all records at once
result_list = neo4j_query(query)

# Create DataFrame using the list of dictionaries
df = pd.DataFrame(result_list)



In [155]:
#---------------------------------- texte_2 Import the author and the article into the graph
authors = ["Shabaz Sultan", "Sapna Devi", "Scott N. Mueller", "Johannes Textor"]

title = "A parallelized cellular Potts model that enables simulations at tissue scale"

neo4j_query("""

MERGE (b:Article {title: $title})

WITH b

UNWIND $authors AS authorName

MERGE (a:Author {name: authorName})

MERGE (a)-[:WROTE]->(b)

""", {'title': title, 'authors': authors})

print("Authors and article imported into the graph")




Authors and article imported into the graph


In [158]:
# Import the sentences and mentioned entities
neo4j_query("""
MATCH (a:Article)
UNWIND $data as row
MERGE (s:Sentence {id: row.text_sha256})
SET s.text = row.text
MERGE (a)-[:HAS_SENTENCE]->(s)
WITH s, row.entities as entities
UNWIND entities as entity
MERGE (e:Entity {id: entity.entity_id})
ON CREATE SET e.other_ids = entity.other_ids,
  e.name = entity.entity,
  e.type = entity.entity_type
MERGE (s)-[m:MENTIONS]->(e)
ON CREATE SET m.count = 1
ON MATCH SET m.count = m.count + 1
""", {'data': parsed_entities})



[]

In [159]:
#-------------------------------------text_2 Import the sentences and mentioned entities
neo4j_query("""
MATCH (a:Article)
UNWIND $data as row
MERGE (s:Sentence {id: row.text_sha256})
SET s.text = row.text
MERGE (a)-[:HAS_SENTENCE]->(s)
WITH s, row.entities as entities
UNWIND entities as entity
MERGE (e:Entity {id: entity.entity_id})
ON CREATE SET e.other_ids = entity.other_ids,
  e.name = entity.entity,
  e.type = entity.entity_type
MERGE (s)-[m:MENTIONS]->(e)
ON CREATE SET m.count = 1
ON MATCH SET m.count = m.count + 1
""", {'data': parsed_entities1})

[]

In [165]:
# example application 1 : search engine
neo4j_query("""
MATCH (e:Entity)<-[:MENTIONS]-(s:Sentence)
WHERE e.name = "autoimmune diseases"
RETURN s.text as result
""")

[{'result': 'These cells, later found to be hematopoietic stem cells, have been used successfully to treat leukemia and some severe autoimmune diseases called bone marrow transplants.'}]

In [166]:
# example application 2 : co-occurrence
neo4j_query("""
MATCH (e1:Entity)<-[:MENTIONS]-()-[:MENTIONS]->(e2:Entity)
WHERE id(e1) < id(e2)
RETURN e1.name as entity1, e2.name as entity2, count(*) as 
cooccurrence
ORDER BY cooccurrence
DESC LIMIT 3
""")

[{'entity1': 'collagen', 'entity2': 'fibroblasts', 'cooccurrence': 6},
 {'entity1': 'mesenchymal stem cells',
  'entity2': 'collagen',
  'cooccurrence': 4},
 {'entity1': 'mesenchymal stem cells',
  'entity2': 'fibroblasts',
  'cooccurrence': 4}]

In [173]:

# example application 3 : author expertise
neo4j_query("""
MATCH (a:Author)-[:WROTE]->()-[:HAS_SENTENCE]->()-[:MENTIONS]-
>(e:Entity)
RETURN a.name as author, e.name as entity, count(*) as count
ORDER BY count DESC
LIMIT 5
""")

[{'author': 'Sapna Devi', 'entity': 'GPU', 'count': 20},
 {'author': 'Scott N. Mueller', 'entity': 'GPU', 'count': 20},
 {'author': 'Shabaz Sultan', 'entity': 'GPU', 'count': 20},
 {'author': 'Mohammadreza Ahmadi', 'entity': 'GPU', 'count': 20},
 {'author': 'Johannes Textor', 'entity': 'GPU', 'count': 20}]

In [164]:
#-------------------------------------text_2
# example application 1 : search engine
# example application 1 : search engine
neo4j_query("""
MATCH (e:Entity)<-[:MENTIONS]-(s:Sentence)
WHERE e.name = "single cells"
RETURN s.text as result
""")


[{'result': 'When modelling tissue environments, many researchers wish to consider both he complex morphology of single cells, and the behavior of these cells as part of larger collectives.'}]

In [171]:
# texte_2
# example co-occurrence
neo4j_query("""
MATCH (s:Sentence1)-[:MENTIONS]->(e1:Entity)
MATCH (s:Sentence1)-[:MENTIONS]->(e2:Entity)
WHERE id(e1) < id(e2)
RETURN e1.name as entity1, e2.name as entity2, count(*) as cooccurrence
ORDER BY cooccurrence DESC LIMIT 3
""")


[]

In [172]:
# tete_2
# example author expertise
neo4j_query("""
MATCH (a:Author)-[:WROTE]->()-[:HAS_SENTENCE]->()-[:MENTIONS]-
>(e:Entity)
RETURN a.name as author, e.name as entity, count(*) as count
ORDER BY count DESC
LIMIT 5
""")

[{'author': 'Sapna Devi', 'entity': 'GPU', 'count': 20},
 {'author': 'Scott N. Mueller', 'entity': 'GPU', 'count': 20},
 {'author': 'Shabaz Sultan', 'entity': 'GPU', 'count': 20},
 {'author': 'Mohammadreza Ahmadi', 'entity': 'GPU', 'count': 20},
 {'author': 'Johannes Textor', 'entity': 'GPU', 'count': 20}]

In [ ]:
from transformers import cached_path

cached_path("https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-finetuned-squad-pytorch_model.bin", cache_dir=None)

Downloading: 100%|██████████| 1.34G/1.34G [14:38<00:00, 1.53MB/s]


'C:\\Users\\Fredy Rodriguez/.cache\\huggingface\\transformers\\ca2ac20761877486c1e2204d99653106b9adacf9a5eb18ec71b41d2dbef42103.2db7ae79c41a184c87600faabafa1369db2b16457723fd154ca3b436c4172807'

In [ ]:
#deleting the contents of the cache directory.
import shutil
import os

cache_dir = os.path.expanduser("~/.cache/huggingface/transformers")
shutil.rmtree(cache_dir, ignore_errors=True)


In [174]:
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

# Load pre-trained model and tokenizer
model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Define the relations you want to extract
relations = ['associated', 'interacts']

# Create a RelationExtractor instance
extractor = RelationExtractor(model, tokenizer, relations)


In [176]:
#------------------------------------text_1
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

# Load pre-trained model and tokenizer
model1 = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer1 = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Define the relations you want to extract
relations1 = ['associated', 'interacts']

# Create a RelationExtractor instance
extractor1 = RelationExtractor(model1, tokenizer1, relations1)

In [177]:
%%time
#Candidate sentence where there is more than a single entity
present_candidates = [s for s in parsed_entities if (s.get('entities')) and (len(s['entities']) > 1)]
print(present_candidates)
predicted_rels = []

for c in present_candidates:
    combinations = itertools.combinations([{'name': x['entity'], 'id': x['entity_id']} for x in c['entities']], 2)
    
    for combination in list(combinations):
        try:
            ranked_rels = extractor.rank(text=c['text'].replace(",", ""), head=combination[0]['name'], tail=combination[1]['name'])
            
            # Define threshold for the most probable relation
            if ranked_rels[0][1] > 0.85:
                predicted_rels.append({
                    'head': combination[0]['id'],
                    'tail': combination[1]['id'],
                    'type': ranked_rels[0][0],
                    'source': c['text_sha256']
                })
        except Exception as e:
            # Handle exceptions appropriately
            pass

[{'entities': [{'entity_id': 'people', 'other_ids': ['NCBITaxon:9606'], 'entity_type': 'species', 'entity': 'people'}, {'entity_id': 'skin diseases', 'other_ids': ['mesh:D012871'], 'entity_type': 'disease', 'entity': 'skin diseases'}, {'entity_id': 'chronic wounds', 'other_ids': ['mesh:D002908'], 'entity_type': 'disease', 'entity': 'chronic wounds'}, {'entity_id': 'non-healing and diabetic ulcers', 'other_ids': ['mesh:D003668'], 'entity_type': 'disease', 'entity': 'non-healing and diabetic ulcers'}], 'text': 'Many people with skin diseases such as chronic wounds, non-healing and diabetic ulcers need reconstruction and regeneration of their skin.', 'text_sha256': '95be1713dbb23e86525959e49ea5a196b49f3bbc1ce8c6cbfa373f176c9d39ae'}, {'entities': [{'entity_id': 'pluripotent', 'other_ids': ['CUI-less'], 'entity_type': 'cell_line', 'entity': 'pluripotent'}, {'entity_id': 'pluripotent stem cells', 'other_ids': ['CUI-less'], 'entity_type': 'cell_type', 'entity': 'pluripotent stem cells'}, {'en

In [179]:
print("Predicted relations:")

Predicted relations:


In [178]:
#------------------------------------text_2
#Candidate sentence where there is more than a single entity
present_candidates1 = [s for s in parsed_entities1 if (s.get('entities')) and (len(s['entities']) > 1)]
print(present_candidates1)
predicted_rels1 = []

for c in present_candidates1:
    combinations1 = itertools.combinations([{'name': x['entity'], 'id': x['entity_id']} for x in c['entities']], 2)
    
    for combination1 in list(combinations1):
        try:
            ranked_rels1 = extractor.rank(text=c['text'].replace(",", ""), head=combination1[0]['name'], tail=combination1[1]['name'])
            
            # Define threshold for the most probable relation
            if ranked_rels1[0][1] > 0.85:
                predicted_rels1.append({
                    'head': combination1[0]['id'],
                    'tail': combination1[1]['id'],
                    'type': ranked_rels1[0][0],
                    'source': c['text_sha256']
                })
        except Exception as e:
            # Handle exceptions appropriately
            pass

[{'entities': [{'entity_id': 'tumor', 'other_ids': ['mesh:D009369'], 'entity_type': 'disease', 'entity': 'tumor'}, {'entity_id': 'tumors', 'other_ids': ['mesh:D009369'], 'entity_type': 'disease', 'entity': 'tumors'}, {'entity_id': 'tumor', 'other_ids': ['mesh:D009369'], 'entity_type': 'disease', 'entity': 'tumor'}], 'text': ':13 Assessment of tumor growth behavior and invasive patterns of tumors likewise changes as a function of dimension, resulting in the use of 3D CPM simulations in the tumor modelling field as well.?', 'text_sha256': '6402458a476479be9942d8ea4c6a13d914e6f4ee3e0207dd0979cbc470f3b351'}, {'entities': [{'entity_id': 'GPUs', 'other_ids': ['CUI-less'], 'entity_type': 'DNA', 'entity': 'GPUs'}, {'entity_id': 'GPU', 'other_ids': ['CUI-less'], 'entity_type': 'DNA', 'entity': 'GPU'}], 'text': 'propose a similar scheme for GPUs:! because the sections are much smaller than a cell on the GPU, they use a per-cell locking mechanism to prevent simultaneous updates of a cell.', 'text

In [180]:
print(predicted_rels)

[{'head': 'skin diseases', 'tail': 'chronic wounds', 'type': 'associated', 'source': '95be1713dbb23e86525959e49ea5a196b49f3bbc1ce8c6cbfa373f176c9d39ae'}, {'head': 'hematopoietic stem cells', 'tail': 'leukemia', 'type': 'associated', 'source': '9785b08aa0fecd2e0a28784a7f550f08201c5fafed3e2fd72cb3e3c4c0faf19a'}, {'head': 'hematopoietic stem cells', 'tail': 'autoimmune diseases', 'type': 'associated', 'source': '9785b08aa0fecd2e0a28784a7f550f08201c5fafed3e2fd72cb3e3c4c0faf19a'}, {'head': 'leukemia', 'tail': 'autoimmune diseases', 'type': 'associated', 'source': '9785b08aa0fecd2e0a28784a7f550f08201c5fafed3e2fd72cb3e3c4c0faf19a'}, {'head': 'Mesenchymal stem cells', 'tail': 'targeted', 'type': 'interacts', 'source': '14158336512ad4f512d796f95bed14dab9902aa06c9b2d352216bb51fb7e5df8'}, {'head': 'collagenase enzymes', 'tail': 'fibroblasts', 'type': 'interacts', 'source': '50d0f4e2ae5e02d9618325ae8cc864b9208eaf9e34c3ac944ba073e058ea1633'}, {'head': 'engineered stem cell', 'tail': 'inflammatory c

In [102]:
print(predicted_rels1)

[]


In [181]:
# Store the extracted relationships in Neo4j
neo4j_query("""
UNWIND $data as row
MATCH (source:Entity {id: row.head})
MATCH (target:Entity {id: row.tail})
MATCH (text:Sentence {id: row.source})
MERGE (source)-[:REL]->(r:Relation {type: row.type})-[:REL]->(target)
MERGE (text)-[:MENTIONS]->(r)
""", {'data': predicted_rels})
print("done")

done


In [182]:
#------------------------------------text_2
# Store the extracted relationships in Neo4j
neo4j_query("""
UNWIND $data as row
MATCH (source:Entity {id: row.head})
MATCH (target:Entity {id: row.tail})
MATCH (text:Sentence {id: row.source})
MERGE (source)-[:REL]->(r:Relation {type: row.type})-[:REL]->(target)
MERGE (text)-[:MENTIONS]->(r)
""", {'data': predicted_rels1})
print("done")

done


In [183]:
# Examine the extracted relationships
neo4j_query("""
MATCH (s:Entity)-[:REL]->(r:Relation)-[:REL]->(t:Entity), (r)<-[:MENTIONS]-(st:Sentence)
RETURN s.name as source_entity, t.name as target_entity, r.type as type, st.text as source_text
""")
print("done")

done


In [184]:
#------------------------------------text_2
# Examine the extracted relationships
neo4j_query("""
MATCH (s:Entity)-[:REL]->(r:Relation)-[:REL]->(t:Entity), (r)<-[:MENTIONS]-(st:Sentence)
RETURN s.name as source_entity, t.name as target_entity, r.type as type, st.text as source_text
""")
print("done")

done


Extration des données depuis neo4j vers une base de données Flask

In [186]:
!pip install flask


     ------------------------------------ 101.8/101.8 kB 649.5 kB/s eta 0:00:00
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached Werkzeug-2.2.3-py3-none-any.whl (233 kB)


In [6]:
#importing flask module in the project is mandatory
from flask import Flask, jsonify
from neo4j import GraphDatabase

In [10]:
# installing the required libraries
!pip install requests


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Fredy Rodriguez\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [11]:
!pip install --upgrade pip

  Using cached pip-23.3.2-py3-none-any.whl.metadata (3.5 kB)
Using cached pip-23.3.2-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\Fredy Rodriguez\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Fredy Rodriguez\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [14]:
# importing the requests library
from flask import Flask, jsonify, render_template

# creating a Flask app
app = Flask(__name__)

# Your neo4j_query function
def get_neo4j_session():
    uri = 'bolt://3.82.231.5:7687'
    user = 'neo4j'
    password = 'silk-merchant-quiets'

    driver = GraphDatabase.driver(uri, auth=(user, password))
    return driver.session()

# Endpoint to get all articles and their authors
@app.route('/api/authors', methods=['GET'])
def get_authors():
    authors = []
    with get_neo4j_session() as session:
        result = session.run("""
            MATCH (a:Author)-[:WROTE]->(b:Article)
            RETURN b.title AS article, b.urlToArticle AS urlToArticle, COLLECT(a.name) AS authors
        """)
        authors = [{'article': record['article'], 'urlToArticle': record['urlToArticle'], 'authors': record['authors']} for record in result]
    return jsonify(authors)


# Endpoint to get all articles and their authors from HTML
@app.route('/auteurs')
def index():
    return render_template('authors.html')

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jan/2024 13:30:03] "GET /auteurs HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 13:30:03] "GET /style.js HTTP/1.1Failed to write data to connection IPv4Address(('3.82.231.5', 7687)) (ResolvedIPv4Address(('3.82.231.5', 7687)))
" 404 -
127.0.0.1 - - [12/Jan/2024 13:30:04] "GET /api/authors HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 13:30:26] "GET /api/authors HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 13:30:34] "GET /api/authors HTTP/1.1" 200 -
Failed to write data to connection IPv4Address(('3.82.231.5', 7687)) (ResolvedIPv4Address(('3.82.231.5', 7687)))
Failed to write data to connection IPv4Address(('3.82.231.5', 7687)) (ResolvedIPv4Address(('3.82.231.5', 7687)))
Failed to write data to connection IPv4Address(('3.82.231.5', 7687)) (ResolvedIPv4Address(('3.82.231.5', 7687)))
127.0.0.1 - - [12/Jan/2024 13:32:12] "GET /api/authors HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 13:32:36] "GET /api/authors HTTP/1.1" 200 -
